# KPM.jl Workflow Test - DOS Computation

Tests actual KPM workflow: computing DOS for a simple Hermitian Hamiltonian

Open in Colab: https://colab.research.google.com/github/Pixley-Research-Group-in-CMT/KPM.jl/blob/colab-test/colab_workflow_test.ipynb

In [ ]:
# === Cell 1: Install Julia ===
!curl -fsSL https://install.julialang.org | sh -s -- --yes
!pip install julia
import os
os.environ['PATH'] += ':/root/.juliaup/bin'
!julia --version

In [ ]:
# === Cell 2: Clone and setup KPM.jl ===
%cd /content
!rm -rf KPM.jl
!git clone https://github.com/Pixley-Research-Group-in-CMT/KPM.jl.git
%cd KPM.jl
!git log --oneline -3

In [ ]:
# === Cell 3: Install dependencies ===
import subprocess
subprocess.run(['julia', '-e', '''
using Pkg
Pkg.add("CUDA")
Pkg.develop(path=".")
Pkg.instantiate()
println("Dependencies installed")
'''], check=True)

In [ ]:
# === Cell 4: Run DOS computation test ===
test_code = '''
using KPM
using CUDA
using SparseArrays
using LinearAlgebra

println("=== KPM.jl Workflow Test ===")
println("CUDA functional: ", CUDA.functional())
println("CUDA has_cuda: ", CUDA.has_cuda())

# Create a simple 1D tight-binding Hamiltonian (Hermitian)
n = 100
t = 1.0
H = spdiagm(-1 => fill(-t, n-1), 0 => fill(0.0, n), 1 => fill(-t, n-1))
println("Hamiltonian size: ", size(H))
println("Is Hermitian: ", ishermitian(H))

# Test CPU path first
println("\n--- CPU Test ---")
try
    H_cpu = KPM.maybe_to_device(Matrix(H))
    println("CPU maybe_to_device: OK (type=", typeof(H_cpu), ")")
    
    # Compute DOS with KPM
    nc = 100  # number of Chebyshev moments
    ket = zeros(n)
    ket[50] = 1.0  # local probe at center
    
    mu = KPM.kpm_moments(H_cpu, ket, nc)
    println("KPM moments computed: ", length(mu), " moments")
    
    # Reconstruct DOS
    Es, dos = KPM.kpm_density(mu, a=0.0, b=4.0)
    println("DOS computed: ", length(Es), " energy points")
    println("DOS integral: ", sum(dos) * (Es[2]-Es[1]))
    
catch e
    println("CPU ERROR: ", e)
    println("CPU stacktrace:")
    Base.show_backtrace(stdout, catch_backtrace())
end

# Test GPU path if available
println("\n--- GPU Test ---")
if CUDA.functional()
    try
        H_gpu = KPM.maybe_to_device(H)
        println("GPU maybe_to_device: OK (type=", typeof(H_gpu), ")")
        
        ket_gpu = KPM.maybe_to_device(ket)
        println("GPU vector transfer: OK (type=", typeof(ket_gpu), ")")
        
        # Try KPM on GPU
        mu_gpu = KPM.kpm_moments(H_gpu, ket_gpu, nc)
        println("GPU KPM moments: OK (", length(mu_gpu), " moments)")
        
        Es_gpu, dos_gpu = KPM.kpm_density(mu_gpu, a=0.0, b=4.0)
        println("GPU DOS computed: OK")
        
    catch e
        println("GPU ERROR: ", e)
        println("GPU stacktrace:")
        Base.show_backtrace(stdout, catch_backtrace())
    end
else
    println("GPU not available, skipping GPU tests")
end

println("\n=== Test Complete ===")
'''

import subprocess
result = subprocess.run(['julia', '-e', test_code], capture_output=True, text=True)

output = result.stdout
if result.stderr:
    output += "\n\nSTDERR:\n" + result.stderr

print(output)

# Save to file
with open('/content/workflow_test_output.txt', 'w') as f:
    f.write(output)

In [ ]:
# === Cell 5: Post results to GitHub PR ===
# Get token from https://github.com/settings/tokens (classic, repo scope)
GITHUB_TOKEN = ""  # <-- paste your token here
PR_NUMBER = 19

import requests
from datetime import datetime

with open('/content/workflow_test_output.txt', 'r') as f:
    test_output = f.read()

comment = f"""
## KPM.jl Workflow Test Results (DOS Computation)

**Time:** {datetime.now().isoformat()}  
**Platform:** Google Colab T4 GPU

### Output
```
{test_output}
```
"""

if GITHUB_TOKEN:
    url = f"https://api.github.com/repos/Pixley-Research-Group-in-CMT/KPM.jl/issues/{PR_NUMBER}/comments"
    headers = {"Authorization": f"token {GITHUB_TOKEN}", "Accept": "application/vnd.github.v3+json"}
    r = requests.post(url, headers=headers, json={"body": comment})
    print(f"Posted: {r.status_code}")
    print(f"URL: {r.json().get('html_url', r.text)}")
else:
    print("No token - copy output manually to PR")
    print(comment)